# Nanopore Sequencing for ML Engineers

Welcome to genomics! This notebook will teach you nanopore sequencing concepts through the lens of machine learning and data science.

## What You'll Learn

1. **What is nanopore sequencing?** (Think: streaming data with noise)
2. **Working with biological sequences** (Think: variable-length categorical sequences)
3. **Sequence quality and error patterns** (Think: noisy labels and systematic biases)
4. **Genomics-specific metrics** (Think: domain-specific evaluation metrics)
5. **How this relates to ML problems** (Think: sequence modeling, error correction, classification)

## Our Dataset

We're using a fantastic real-world dataset from Zenodo:
- **E. coli K-12 MG1655** nanopore sequencing data (FASTQ format)
- **Basecalled with Bonito** (high-quality basecaller)
- **Oxford Nanopore MinION** sequencer data
- **Quality scores included** - perfect for ML applications!

**Why E. coli K-12 MG1655?**
- The "MNIST of genomics" - most studied bacterial strain
- Small genome (~4.6 million base pairs) - manageable for learning
- Single circular chromosome - simple structure
- Reference genome available from the same dataset
- Authentic nanopore error patterns for ML training

**Dataset advantages:**
✅ FASTQ format (sequences + quality scores)  
✅ Real MinION data (authentic error patterns)  
✅ Matched reference genome  
✅ High-quality basecalling (Bonito)  
✅ Perfect for ML error correction models  

Let's dive in! 🧬


## 1. Understanding Nanopore Sequencing

**What is nanopore sequencing?**

Imagine you have a tiny hole (nanopore) in a membrane. DNA molecules pass through this hole one at a time. As each nucleotide (A, T, G, C) passes through, it changes the electrical current in a characteristic way.

**ML Analogy:**
- Think of it as **streaming sensor data**
- Each nucleotide produces a different **electrical signal**
- You need to **decode** these signals back to DNA sequence
- It's a **sequence-to-sequence** problem with noise

**Key characteristics:**
- **Long reads**: 1,000 - 100,000+ base pairs (vs. 150-300 for Illumina)
- **Higher error rate**: ~10-15% vs. ~0.1% for Illumina
- **Real-time**: Data comes off the sequencer live
- **Single molecule**: Each read is from one DNA molecule

This makes it perfect for:
- Genome assembly (long reads span repetitive regions)
- Structural variant detection
- Real-time analysis (pathogen identification)

But challenging for:
- High-accuracy applications
- Small variant detection
- Cost-sensitive projects


In [ ]:
# Setup and imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
import gzip
from pathlib import Path
from collections import Counter, defaultdict
import json

# ML libraries we'll use later
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

print("✅ Libraries imported successfully!")
print("📂 Current working directory:", Path.cwd())

# Check if data exists
data_file = Path("../data/raw/ecoli_nanopore_reads.fastq.gz")
if data_file.exists():
    print(f"✅ Data file found: {data_file}")
    print(f"   File size: {data_file.stat().st_size / (1024*1024):.1f} MB")
    print("📊 This is FASTQ format - sequences with quality scores!")
else:
    print("❌ Data file not found!")
    print("   Run this first: uv run python scripts/download_data.py")


## 2. Loading and Exploring Sequence Data

In bioinformatics, we work with several file formats:
- **FASTA**: Just sequences (like text files)
- **FASTQ**: Sequences + quality scores (like labeled data)
- **SAM/BAM**: Aligned sequences (like predictions with ground truth)

Our data is in FASTA format - think of it as a collection of variable-length strings where each character is one of four categories: A, T, G, C.

**ML Perspective:**
- Each read = one training sample
- Each nucleotide = one feature/token
- Variable length sequences (like sentences in NLP)
- Categorical data (4 classes: A, T, G, C)


In [ ]:
# Load a sample of reads for exploration
print("🧬 Loading nanopore reads...")

# Load first 10,000 reads for interactive exploration
reads_data = []
max_reads = 10000  # Limit for notebook performance

with gzip.open(data_file, "rt") as handle:
    for i, record in enumerate(SeqIO.parse(handle, "fastq")):
        if i >= max_reads:
            break
            
        # FASTQ format gives us both sequence and quality scores!
        reads_data.append({
            'read_id': record.id,
            'sequence': str(record.seq),
            'length': len(record.seq),
            'quality_scores': record.letter_annotations['phred_quality'],
            'mean_quality': sum(record.letter_annotations['phred_quality']) / len(record.letter_annotations['phred_quality'])
        })
        
        if (i + 1) % 2000 == 0:
            print(f"  Loaded {i + 1:,} reads...")

print(f"✅ Loaded {len(reads_data):,} reads for analysis")

# Quick peek at the data structure
print(f"\n📊 Data structure:")
print(f"  Type: {type(reads_data)}")
print(f"  First read keys: {list(reads_data[0].keys())}")
print(f"\n🔍 First few reads:")

for i in range(3):
    read = reads_data[i]
    sequence_preview = read['sequence'][:50] + "..." if len(read['sequence']) > 50 else read['sequence']
    print(f"  Read {i+1}: ID={read['read_id'][:20]}..., Length={read['length']:,}bp")
    print(f"         Sequence={sequence_preview}")
    print(f"         Mean Quality: {read['mean_quality']:.1f} (Phred score)")
    print()

print("📊 ✅ Quality scores available - FASTQ format with Phred scores!")
print("💡 Quality scores help identify reliable vs. unreliable bases")
print("   Higher scores = more confident base calls")
print("   This is crucial for ML error correction models!")


## 3. Understanding Read Length Distributions

Read length is one of the most important characteristics of sequencing data.

**Why length matters:**
- **Assembly**: Longer reads can span repetitive regions that short reads cannot
- **Mapping**: Longer reads map more uniquely to genomes
- **Errors**: Longer reads have more errors in absolute terms, but similar error rates
- **Computational cost**: Longer sequences require more processing time

**ML Perspective:**
- Like having variable-length input sequences in NLP
- Need to handle different sequence lengths in models
- May need padding, truncation, or specialized architectures (RNNs, Transformers)
- Length can be a useful feature for downstream tasks


In [ ]:
# Analyze read length distribution
lengths = [read['length'] for read in reads_data]

# Calculate key statistics
def calculate_n50(lengths):
    """
    N50: The length such that 50% of all bases are in reads >= this length.
    
    Different from median! N50 is weighted by read length.
    Think of it as: "Half of the sequenced DNA comes from reads this long or longer"
    """
    sorted_lengths = sorted(lengths, reverse=True)
    total_bases = sum(sorted_lengths)
    cumulative_bases = 0
    
    for length in sorted_lengths:
        cumulative_bases += length
        if cumulative_bases >= total_bases / 2:
            return length
    return 0

# Basic statistics
stats = {
    'count': len(lengths),
    'total_bases': sum(lengths),
    'mean': np.mean(lengths),
    'median': np.median(lengths),
    'std': np.std(lengths),
    'min': min(lengths),
    'max': max(lengths),
    'n50': calculate_n50(lengths)
}

print("📊 READ LENGTH STATISTICS")
print("=" * 40)
print(f"Total reads:      {stats['count']:,}")
print(f"Total bases:      {stats['total_bases']:,} ({stats['total_bases']/1e6:.1f}M)")
print(f"Mean length:      {stats['mean']:,.0f} bp")
print(f"Median length:    {stats['median']:,.0f} bp")
print(f"Standard dev:     {stats['std']:,.0f} bp")
print(f"N50:              {stats['n50']:,.0f} bp")
print(f"Range:            {stats['min']:,} - {stats['max']:,} bp")

# Percentiles (like quantiles in data science)
percentiles = [5, 25, 75, 95]
print(f"\nPercentiles:")
for p in percentiles:
    value = np.percentile(lengths, p)
    print(f"  {p}th percentile: {value:,.0f} bp")


In [ ]:
# Visualize read length distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Histogram of read lengths
axes[0, 0].hist(lengths, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].axvline(stats['mean'], color='red', linestyle='--', label=f'Mean: {stats["mean"]:,.0f}')
axes[0, 0].axvline(stats['median'], color='orange', linestyle='--', label=f'Median: {stats["median"]:,.0f}')
axes[0, 0].axvline(stats['n50'], color='green', linestyle='--', label=f'N50: {stats["n50"]:,.0f}')
axes[0, 0].set_xlabel('Read Length (bp)')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('Read Length Distribution')
axes[0, 0].legend()
axes[0, 0].set_yscale('log')  # Log scale to see the full distribution

# 2. Cumulative distribution (like CDF in statistics)
sorted_lengths = sorted(lengths, reverse=True)
cumulative_bases = np.cumsum(sorted_lengths)
axes[0, 1].plot(range(len(cumulative_bases)), cumulative_bases, linewidth=2)
axes[0, 1].axhline(stats['total_bases'] / 2, color='red', linestyle='--', 
                   label='50% of bases')
axes[0, 1].set_xlabel('Read Rank (longest to shortest)')
axes[0, 1].set_ylabel('Cumulative Bases')
axes[0, 1].set_title('Cumulative Base Distribution')
axes[0, 1].legend()

# 3. Box plot with quartiles
bp = axes[1, 0].boxplot([lengths], patch_artist=True, 
                        boxprops=dict(facecolor='lightblue'))
axes[1, 0].set_ylabel('Read Length (bp)')
axes[1, 0].set_title('Read Length Box Plot')
axes[1, 0].set_xticks([1])
axes[1, 0].set_xticklabels(['All Reads'])

# Add text annotations
quartiles = np.percentile(lengths, [25, 50, 75])
axes[1, 0].text(1.3, quartiles[1], f'Median: {quartiles[1]:,.0f}', 
                verticalalignment='center')

# 4. Length bins (like creating features for ML)
length_bins = [0, 1000, 5000, 10000, 20000, 50000, float('inf')]
length_labels = ['<1kb', '1-5kb', '5-10kb', '10-20kb', '20-50kb', '>50kb']

# Bin the data
binned_counts = []
for i in range(len(length_bins)-1):
    count = sum(1 for l in lengths if length_bins[i] <= l < length_bins[i+1])
    binned_counts.append(count)

# Plot binned data
bars = axes[1, 1].bar(length_labels, binned_counts, 
                      color=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#ff99cc', '#c2c2f0'])
axes[1, 1].set_ylabel('Count')
axes[1, 1].set_title('Read Length Bins')
axes[1, 1].tick_params(axis='x', rotation=45)

# Add count labels on bars
for bar, count in zip(bars, binned_counts):
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + max(binned_counts)*0.01,
                    f'{count:,}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n💡 ML Engineering Observations:")
print("• Wide distribution suggests you'll need to handle variable sequence lengths")
print("• Long tail distribution - most reads are short, few are very long")
print("• This is similar to document length distributions in NLP")
print("• Consider length-based sampling or stratification for training")


## 4. Nucleotide Composition and GC Content

DNA is made of four nucleotides: A (Adenine), T (Thymine), G (Guanine), C (Cytosine)

**Key concepts:**
- **Base pairing**: A pairs with T, G pairs with C (in double-stranded DNA)
- **GC content**: Percentage of bases that are G or C
- **GC bias**: Deviation from expected 50% (important for sequencing quality)

**Why GC content matters:**
- **Stability**: G-C pairs have 3 hydrogen bonds vs 2 for A-T (stronger)
- **Melting temperature**: Higher GC = higher temperature needed to separate strands
- **Sequencing bias**: Some sequencers struggle with very high or low GC regions
- **Species identification**: Different organisms have characteristic GC contents

**ML Perspective:**
- Like analyzing character frequencies in text
- GC content can be a useful feature for classification
- Compositional bias can affect model training
- Similar to analyzing word frequencies or n-gram distributions


In [ ]:
# Analyze nucleotide composition
print("🧮 Analyzing nucleotide composition...")

# Count all nucleotides across all reads
all_nucleotides = Counter()
gc_contents = []
read_compositions = []

for read in reads_data:
    sequence = read['sequence'].upper()
    
    # Count nucleotides in this read
    read_counts = Counter(sequence)
    all_nucleotides.update(read_counts)
    
    # Calculate GC content for this read
    total_bases = sum(count for base, count in read_counts.items() if base in 'ATGC')
    if total_bases > 0:
        gc_count = read_counts.get('G', 0) + read_counts.get('C', 0)
        gc_content = (gc_count / total_bases) * 100
        gc_contents.append(gc_content)
        
        # Store composition for this read
        composition = {
            'A': (read_counts.get('A', 0) / total_bases) * 100,
            'T': (read_counts.get('T', 0) / total_bases) * 100,
            'G': (read_counts.get('G', 0) / total_bases) * 100,
            'C': (read_counts.get('C', 0) / total_bases) * 100,
            'GC': gc_content
        }
        read_compositions.append(composition)

# Overall composition
total_valid_bases = sum(all_nucleotides[base] for base in 'ATGC')
overall_composition = {
    base: (all_nucleotides[base] / total_valid_bases) * 100 
    for base in 'ATGC'
}
overall_gc = overall_composition['G'] + overall_composition['C']

print("📊 OVERALL NUCLEOTIDE COMPOSITION")
print("=" * 45)
for base in 'ATGC':
    print(f"{base}: {overall_composition[base]:6.2f}% ({all_nucleotides[base]:,} bases)")

print(f"\nGC content: {overall_gc:.2f}%")
print(f"AT content: {100 - overall_gc:.2f}%")

# GC content statistics
gc_stats = {
    'mean': np.mean(gc_contents),
    'median': np.median(gc_contents),
    'std': np.std(gc_contents),
    'min': min(gc_contents),
    'max': max(gc_contents)
}

print(f"\n📊 GC CONTENT STATISTICS (per read)")
print("=" * 45)
for stat, value in gc_stats.items():
    print(f"{stat.capitalize():>8}: {value:6.2f}%")

# Check for unusual bases (sequencing errors or ambiguous calls)
unusual_bases = {base: count for base, count in all_nucleotides.items() 
                 if base not in 'ATGC' and count > 0}

if unusual_bases:
    print(f"\n⚠️  UNUSUAL BASES DETECTED:")
    for base, count in unusual_bases.items():
        pct = (count / sum(all_nucleotides.values())) * 100
        print(f"   {base}: {count:,} occurrences ({pct:.3f}%)")
        
    print("   Note: 'N' usually means ambiguous/unknown base")


In [ ]:
# Visualize nucleotide composition and GC content
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Overall nucleotide composition (bar chart)
bases = ['A', 'T', 'G', 'C']
percentages = [overall_composition[base] for base in bases]
colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4']

bars = axes[0, 0].bar(bases, percentages, color=colors, edgecolor='black', alpha=0.8)
axes[0, 0].set_ylabel('Percentage (%)')
axes[0, 0].set_title('Overall Nucleotide Composition')
axes[0, 0].set_ylim(0, max(percentages) * 1.1)

# Add percentage labels on bars
for bar, pct in zip(bars, percentages):
    axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
                    f'{pct:.1f}%', ha='center', va='bottom', fontweight='bold')

# Expected vs observed (E. coli has ~50.8% GC content)
expected_gc = 50.8
axes[0, 0].axhline(y=expected_gc/2, color='red', linestyle='--', alpha=0.5, 
                   label=f'Expected G/C ({expected_gc/2:.1f}%)')
axes[0, 0].axhline(y=(100-expected_gc)/2, color='orange', linestyle='--', alpha=0.5,
                   label=f'Expected A/T ({(100-expected_gc)/2:.1f}%)')
axes[0, 0].legend()

# 2. GC content distribution across reads
axes[0, 1].hist(gc_contents, bins=40, edgecolor='black', alpha=0.7, color='teal')
axes[0, 1].axvline(gc_stats['mean'], color='red', linestyle='--', 
                   label=f'Mean: {gc_stats["mean"]:.1f}%')
axes[0, 1].axvline(expected_gc, color='orange', linestyle='--', 
                   label=f'E. coli expected: {expected_gc}%')
axes[0, 1].set_xlabel('GC Content (%)')
axes[0, 1].set_ylabel('Number of Reads')
axes[0, 1].set_title('GC Content Distribution Across Reads')
axes[0, 1].legend()

# 3. GC content vs Read length (scatter plot)
sample_idx = np.random.choice(len(read_compositions), size=min(2000, len(read_compositions)), replace=False)
sample_gc = [gc_contents[i] for i in sample_idx]
sample_lengths = [lengths[i] for i in sample_idx]

axes[1, 0].scatter(sample_gc, sample_lengths, alpha=0.6, s=10, color='purple')
axes[1, 0].set_xlabel('GC Content (%)')
axes[1, 0].set_ylabel('Read Length (bp)')
axes[1, 0].set_title('GC Content vs Read Length')

# Add correlation coefficient
correlation = np.corrcoef(sample_gc, sample_lengths)[0, 1]
axes[1, 0].text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
                transform=axes[1, 0].transAxes, bbox=dict(boxstyle='round', facecolor='wheat'))

# 4. Composition heatmap (showing variation across reads)
# Sample some reads for visualization
sample_size = min(100, len(read_compositions))
sample_compositions = read_compositions[:sample_size]

# Create matrix for heatmap
comp_matrix = np.array([[comp[base] for base in bases] for comp in sample_compositions])

im = axes[1, 1].imshow(comp_matrix, cmap='viridis', aspect='auto')
axes[1, 1].set_xlabel('Nucleotide')
axes[1, 1].set_ylabel(f'Read Index (sample of {sample_size})')
axes[1, 1].set_title('Composition Heatmap Across Reads')
axes[1, 1].set_xticks(range(len(bases)))
axes[1, 1].set_xticklabels(bases)

# Add colorbar
cbar = plt.colorbar(im, ax=axes[1, 1])
cbar.set_label('Percentage (%)')

plt.tight_layout()
plt.show()

print(f"\n💡 Key Observations:")
print(f"• E. coli expected GC content: ~{expected_gc}%")
print(f"• Observed GC content: {overall_gc:.1f}%")
print(f"• GC bias: {abs(overall_gc - expected_gc):.1f}% deviation")
print(f"• GC-length correlation: {correlation:.3f}")

if abs(overall_gc - expected_gc) > 2:
    print("⚠️  Significant GC bias detected - check data quality")
else:
    print("✅ GC content matches expected values - good data quality")


## 4.5. Quality Score Analysis - FASTQ Advantage!

Since our data is in FASTQ format, we have **quality scores** for each nucleotide! This is huge for ML applications.

**What are quality scores?**
- **Phred scores**: Measure confidence in each base call
- **Scale**: Higher = better (usually 0-40+)
- **Logarithmic**: Q20 = 1% error rate, Q30 = 0.1% error rate, Q40 = 0.01% error rate

**Quality Score Formula:**
```
Q = -10 * log10(P_error)
```
Where P_error is the probability the base call is wrong.

**ML Perspective:**
- Like **confidence scores** in classification models
- Can weight training samples by quality
- Essential for **uncertainty quantification**
- Perfect for building **quality-aware models**
- Similar to attention weights or prediction confidence


In [ ]:
# Analyze quality scores from our FASTQ data
print("📊 Analyzing quality scores...")

# Extract quality data from our reads
all_qualities = []
read_mean_qualities = []
quality_by_position = defaultdict(list)

# Sample reads for position analysis (too memory intensive otherwise)
sample_reads = reads_data[:1000]

for read in sample_reads:
    qualities = read['quality_scores']
    all_qualities.extend(qualities)
    read_mean_qualities.append(read['mean_quality'])
    
    # Quality by position (for first 100bp to see patterns)
    for pos, qual in enumerate(qualities[:100]):
        quality_by_position[pos].append(qual)
# Calculate statistics
quality_stats = {
    'mean': np.mean(all_qualities),
    'median': np.median(all_qualities),
    'std': np.std(all_qualities),
    'min': min(all_qualities),
    'max': max(all_qualities),
    'q10': np.percentile(all_qualities, 10),
    'q25': np.percentile(all_qualities, 25),
    'q75': np.percentile(all_qualities, 75),
    'q90': np.percentile(all_qualities, 90)
}

print("📊 QUALITY SCORE STATISTICS")
print("=" * 45)
print(f"Total bases analyzed:  {len(all_qualities):,}")
print(f"Mean quality:          {quality_stats['mean']:.1f}")
print(f"Median quality:        {quality_stats['median']:.1f}")
print(f"Standard deviation:    {quality_stats['std']:.1f}")
print(f"Range:                 {quality_stats['min']} - {quality_stats['max']}")
print(f"25th-75th percentile:  {quality_stats['q25']:.1f} - {quality_stats['q75']:.1f}")

# Convert to error probabilities for interpretation
mean_error_prob = 10 ** (-quality_stats['mean'] / 10)
median_error_prob = 10 ** (-quality_stats['median'] / 10)

print(f"\n🧮 ERROR RATE INTERPRETATION:")
print(f"Mean error probability:   {mean_error_prob:.4f} ({mean_error_prob*100:.2f}%)")
print(f"Median error probability: {median_error_prob:.4f} ({median_error_prob*100:.2f}%)")

# Quality thresholds commonly used
q10_count = sum(1 for q in all_qualities if q >= 10)
q20_count = sum(1 for q in all_qualities if q >= 20)
q30_count = sum(1 for q in all_qualities if q >= 30)

print(f"\n📏 QUALITY THRESHOLDS:")
print(f"Q≥10 (≤10% error):  {q10_count:,} bases ({q10_count/len(all_qualities)*100:.1f}%)")
print(f"Q≥20 (≤1% error):   {q20_count:,} bases ({q20_count/len(all_qualities)*100:.1f}%)")
print(f"Q≥30 (≤0.1% error): {q30_count:,} bases ({q30_count/len(all_qualities)*100:.1f}%)")


In [ ]:
# Visualize quality scores from our FASTQ data

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Quality score distribution
axes[0, 0].hist(all_qualities, bins=50, edgecolor='black', alpha=0.7, color='skyblue')
axes[0, 0].axvline(quality_stats['mean'], color='red', linestyle='--', 
                   label=f'Mean: {quality_stats["mean"]:.1f}')
axes[0, 0].axvline(quality_stats['median'], color='orange', linestyle='--',
                   label=f'Median: {quality_stats["median"]:.1f}')

# Add quality threshold lines
for q, color in [(10, 'green'), (20, 'blue'), (30, 'purple')]:
    axes[0, 0].axvline(q, color=color, linestyle=':', alpha=0.7, 
                      label=f'Q{q} threshold')

axes[0, 0].set_xlabel('Quality Score (Phred)')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('Quality Score Distribution')
axes[0, 0].legend()
axes[0, 0].set_yscale('log')

# 2. Quality vs Read Length
sample_indices = np.random.choice(len(read_mean_qualities), 
                                size=min(2000, len(read_mean_qualities)), 
                                replace=False)
sample_quals = [read_mean_qualities[i] for i in sample_indices]
sample_lens = [lengths[i] for i in sample_indices]

axes[0, 1].scatter(sample_lens, sample_quals, alpha=0.6, s=10, color='purple')
axes[0, 1].set_xlabel('Read Length (bp)')
axes[0, 1].set_ylabel('Mean Quality Score')
axes[0, 1].set_title('Quality vs Read Length')

# Add correlation
qual_length_corr = np.corrcoef(sample_lens, sample_quals)[0, 1]
axes[0, 1].text(0.05, 0.95, f'Correlation: {qual_length_corr:.3f}', 
                transform=axes[0, 1].transAxes, 
                bbox=dict(boxstyle='round', facecolor='wheat'))

# 3. Quality by position (first 50bp)
positions = sorted(quality_by_position.keys())[:50]  # First 50 positions
mean_quals_by_pos = [np.mean(quality_by_position[pos]) for pos in positions]
std_quals_by_pos = [np.std(quality_by_position[pos]) for pos in positions]

axes[1, 0].plot(positions, mean_quals_by_pos, 'b-', linewidth=2, label='Mean Quality')
axes[1, 0].fill_between(positions, 
                       [m-s for m,s in zip(mean_quals_by_pos, std_quals_by_pos)],
                       [m+s for m,s in zip(mean_quals_by_pos, std_quals_by_pos)],
                       alpha=0.3)
axes[1, 0].set_xlabel('Position in Read')
axes[1, 0].set_ylabel('Quality Score')
axes[1, 0].set_title('Quality Score by Position')
axes[1, 0].legend()
axes[1, 0].axhline(y=20, color='red', linestyle='--', alpha=0.7, label='Q20')
axes[1, 0].axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Q30')

# 4. Error rate visualization
error_rates = [10 ** (-q / 10) for q in all_qualities]
axes[1, 1].hist(error_rates, bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1, 1].set_xlabel('Error Probability')
axes[1, 1].set_ylabel('Count')
axes[1, 1].set_title('Error Rate Distribution')
axes[1, 1].set_xscale('log')
axes[1, 1].set_yscale('log')

plt.tight_layout()
plt.show()

print(f"\n💡 Quality Analysis Insights:")
print(f"• Mean quality: {quality_stats['mean']:.1f} (error rate: {mean_error_prob*100:.2f}%)")
print(f"• Quality-length correlation: {qual_length_corr:.3f}")
print(f"• {q20_count/len(all_qualities)*100:.1f}% of bases are Q≥20 (high quality)")
print(f"• This quality information is gold for ML error correction!")


## 5. Homopolymer Analysis - The Nanopore Challenge

**What are homopolymers?**
Runs of identical bases: AAAAA, TTTTT, GGGGG, CCCCC

**Why do homopolymers matter for nanopore sequencing?**
- **Major error source**: Nanopore sequencers struggle to count identical bases accurately
- **Length ambiguity**: Is it AAAA or AAAAA? Hard to tell from electrical signal
- **ML challenge**: Like trying to count repeated words in noisy speech

**Real-world impact:**
- Gene disruption if homopolymer length is wrong
- Assembly errors in repetitive regions
- False positive/negative variant calls

**ML Perspective:**
- Similar to sequence alignment problems
- Could use attention mechanisms to focus on context
- Error correction models need to handle variable-length insertions/deletions
- Classic sequence labeling problem with systematic noise


In [ ]:
# Analyze homopolymer runs
import re

def find_homopolymers(sequence, min_length=4):
    """Find all homopolymer runs in a sequence"""
    homopolymers = []
    for base in 'ATGC':
        pattern = f'{base}{{{min_length},}}'  # e.g., 'A{4,}' matches AAAA+
        for match in re.finditer(pattern, sequence):
            homopolymers.append({
                'base': base,
                'length': match.end() - match.start(),
                'position': match.start()
            })
    return homopolymers

print("🔄 Analyzing homopolymers...")

# Analyze homopolymers in sample of reads
homopolymer_data = {'A': [], 'T': [], 'G': [], 'C': []}
reads_with_homopolymers = 0
total_homopolymers = 0

# Use subset for performance
sample_reads = reads_data[:2000]

for read in sample_reads:
    homopolymers = find_homopolymers(read['sequence'])
    
    if homopolymers:
        reads_with_homopolymers += 1
        total_homopolymers += len(homopolymers)
        
        for hp in homopolymers:
            homopolymer_data[hp['base']].append(hp['length'])

print(f"📊 HOMOPOLYMER ANALYSIS (sample of {len(sample_reads):,} reads)")
print("=" * 55)
print(f"Reads with homopolymers: {reads_with_homopolymers:,} ({reads_with_homopolymers/len(sample_reads)*100:.1f}%)")
print(f"Total homopolymers found: {total_homopolymers:,}")

print(f"\nPer-base statistics:")
for base in 'ATGC':
    lengths = homopolymer_data[base]
    if lengths:
        print(f"  {base}: {len(lengths):4,} runs, "
              f"avg length: {np.mean(lengths):.1f}, "
              f"max length: {max(lengths):2d}")
    else:
        print(f"  {base}:    0 runs")

# Visualize homopolymer distributions
if total_homopolymers > 0:
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    bases = ['A', 'T', 'G', 'C']
    colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4']
    
    for i, (base, color) in enumerate(zip(bases, colors)):
        ax = axes[i // 2, i % 2]
        lengths = homopolymer_data[base]
        
        if lengths:
            # Histogram of homopolymer lengths
            ax.hist(lengths, bins=range(4, max(lengths)+2), 
                   color=color, alpha=0.7, edgecolor='black')
            ax.set_xlabel('Homopolymer Length')
            ax.set_ylabel('Count')
            ax.set_title(f'{base} Homopolymers (n={len(lengths):,})')
            ax.set_yscale('log')
            
            # Add statistics text
            stats_text = f'Mean: {np.mean(lengths):.1f}\nMax: {max(lengths)}'
            ax.text(0.7, 0.9, stats_text, transform=ax.transAxes,
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        else:
            ax.text(0.5, 0.5, f'No {base} homopolymers\nfound (≥4bp)', 
                   ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{base} Homopolymers')
    
    plt.tight_layout()
    plt.show()
    
    # Combined analysis
    all_lengths = []
    base_labels = []
    for base in bases:
        all_lengths.extend(homopolymer_data[base])
        base_labels.extend([base] * len(homopolymer_data[base]))
    
    if all_lengths:
        plt.figure(figsize=(10, 6))
        
        # Box plot comparison
        plt.subplot(1, 2, 1)
        bp_data = [homopolymer_data[base] for base in bases if homopolymer_data[base]]
        bp_labels = [base for base in bases if homopolymer_data[base]]
        
        if bp_data:
            bp = plt.boxplot(bp_data, labels=bp_labels, patch_artist=True)
            for patch, color in zip(bp['boxes'], colors[:len(bp_data)]):
                patch.set_facecolor(color)
                
        plt.ylabel('Homopolymer Length')
        plt.title('Length Distribution by Base')
        
        # Overall distribution
        plt.subplot(1, 2, 2)
        plt.hist(all_lengths, bins=range(4, max(all_lengths)+2), 
                alpha=0.7, edgecolor='black', color='gray')
        plt.xlabel('Homopolymer Length')
        plt.ylabel('Count')
        plt.title('All Homopolymer Lengths')
        plt.yscale('log')
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n💡 ML Engineering Insights:")
        print(f"• {reads_with_homopolymers/len(sample_reads)*100:.1f}% of reads contain homopolymers ≥4bp")
        print(f"• Average homopolymer length: {np.mean(all_lengths):.1f}bp")
        print(f"• Longest homopolymer: {max(all_lengths)}bp")
        print(f"• This represents the main error-prone regions for nanopore sequencing")
        print(f"• Perfect target for ML error correction models!")
else:
    print("No homopolymers ≥4bp found in sample")


## 🎓 Summary: From Genomics to ML

**Congratulations!** You've just analyzed real nanopore sequencing data. Here's what you learned:

### Genomics Concepts → ML Parallels

| Genomics | ML/Data Science |
|----------|----------------|
| **Read length distribution** | Variable-length sequences (like text) |
| **GC content** | Feature engineering from categorical data |
| **Homopolymer errors** | Systematic noise patterns |
| **N50 metric** | Domain-specific evaluation metrics |
| **Nucleotide composition** | Token/character frequency analysis |

### Key Insights for ML Engineers

1. **Variable-length data**: Like NLP, genomics deals with sequences of different lengths
2. **Systematic errors**: Homopolymers are predictable error sources (perfect for ML!)
3. **Domain knowledge matters**: Understanding biology helps build better models
4. **Quality metrics**: N50, GC content are like accuracy, F1-score for genomics
5. **Real-world data is messy**: Even "high-quality" sequencing has 10-15% error rates

### Next Steps & ML Project Ideas

**🔬 Data Understanding Projects:**
- Compare different organisms' GC content
- Analyze error patterns in more depth
- Build quality control pipelines

**🤖 ML Model Projects:**
- **Quality-aware error correction**: Use quality scores to weight corrections
- **Homopolymer length correction**: Predict true length from context + quality
- **Base calling improvement**: Train models to improve basecalling accuracy  
- **Quality score recalibration**: Improve quality score accuracy
- **Species classification**: Identify organism from read composition
- **Structural variant detection**: Find large genomic rearrangements

**📊 Analysis Extensions:**
- Align reads to reference genome (add mapping data)
- Compare nanopore vs. Illumina sequencing
- Real-time analysis simulation

### Resources for Going Deeper

- **BioPython tutorials**: https://biopython.org/wiki/Category:Tutorial
- **Oxford Nanopore Community**: https://community.nanoporetech.com/
- **Genomics ML papers**: Start with error correction and basecalling models

---

**You're now ready to tackle genomics ML problems!** The biological concepts you've learned here form the foundation for understanding why certain ML approaches work well in genomics and others don't.

Happy coding! 🧬🤖
